In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# --- A. FOCAL LOSS (Mejor que CrossEntropy con pesos simples) ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss) # probabilidad de la clase correcta
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        else:
            return focal_loss.sum()

# --- B. NUEVA ARQUITECTURA (Embudo + GELU + Menor Dropout) ---
class ImprovedMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(ImprovedMLP, self).__init__()
        
        # Arquitectura de "Embudo" para forzar compresión de características
        # 900 -> 512 -> 256 -> 128 -> 2
        
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.drop1 = nn.Dropout(0.3) # Menos agresivo que 0.7
        
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.drop2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.drop3 = nn.Dropout(0.2)
        
        self.output = nn.Linear(128, output_size)
        
    def forward(self, x):
        # Usamos GELU en lugar de ReLU (funciona mejor en NLP/Embeddings)
        x = self.drop1(F.gelu(self.bn1(self.fc1(x))))
        x = self.drop2(F.gelu(self.bn2(self.fc2(x))))
        x = self.drop3(F.gelu(self.bn3(self.fc3(x))))
        x = self.output(x)
        return x

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# --- 0. CARGA Y PREPARACIÓN DE DATOS ---

print("Cargando dataset...")
# Asegúrate de que la ruta sea correcta según tu estructura de carpetas
dataset = pd.read_json("../Datasets/dataset_humor_train_embeddings.json", lines=True)

print("Procesando embeddings...")
# Extraer y apilar los arrays de numpy de las columnas
we_ft = np.stack(dataset['we_ft'].values) # FastText General
we_mx = np.stack(dataset['we_mx'].values) # FastText México
we_es = np.stack(dataset['we_es'].values) # FastText España

# Concatenar los 3 vectores (N, 900)
X_full = np.concatenate([we_ft, we_mx, we_es], axis=1)
Y_full = dataset['klass'].to_numpy()

print(f"Dimensión total de entrada: {X_full.shape}")

# Dividir en Train y Validation
# NOTA: Mantenemos random_state=42 fijo aquí para que todos los modelos del ensamble
# se entrenen y validen con los mismos datos. La "variedad" vendrá de la inicialización de la red.
X_tr, X_val, Y_train, Y_val = train_test_split(
    X_full, Y_full, 
    test_size=0.15, 
    random_state=42, 
    stratify=Y_full # Importante para mantener la proporción de humor
)

print(f"Datos listos -> Train: {X_tr.shape}, Val: {X_val.shape}")

Cargando dataset...
Procesando embeddings...
Dimensión total de entrada: (10400, 900)
Datos listos -> Train: (8840, 900), Val: (1560, 900)


In [7]:
# 1. Función para generar mixup (pon esto fuera de la clase o función)
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# 2. Nueva función de pérdida para mixup
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [14]:
class GatedFusionMLP(nn.Module):
    def __init__(self, output_size=2):
        super(GatedFusionMLP, self).__init__()
        
        # Dimensiones originales de cada vector
        self.dim = 300 
        
        # --- Mecanismo de Atención ---
        # Calcula un "score" de importancia para cada fuente (FT, MX, ES)
        self.attention_net = nn.Sequential(
            nn.Linear(900, 128),
            nn.Tanh(),
            nn.Linear(128, 3), # 3 pesos de salida (uno para cada vector)
            nn.Softmax(dim=1)
        )
        
        # --- Clasificador Principal (ahora recibe 300, no 900) ---
        # Porque haremos una suma ponderada de los vectores
        self.classifier = nn.Sequential(
            nn.Linear(300, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, output_size)
        )
        
    def forward(self, x):
        # x viene con shape (Batch, 900)
        # Separamos los 3 embeddings originales
        ft = x[:, 0:300]
        mx = x[:, 300:600]
        es = x[:, 600:900]
        
        # 1. Calcular pesos de atención
        weights = self.attention_net(x) # (Batch, 3)
        w_ft = weights[:, 0].unsqueeze(1)
        w_mx = weights[:, 1].unsqueeze(1)
        w_es = weights[:, 2].unsqueeze(1)
        
        # 2. Fusión Ponderada (Weighted Average)
        # El vector resultante es una mezcla inteligente de los 3
        fused_vector = (ft * w_ft) + (mx * w_mx) + (es * w_es) # (Batch, 300)
        
        # 3. Clasificación
        out = self.classifier(fused_vector)
        return out

In [8]:
def entrenar_modelo_semilla(seed, X_tr, Y_train, X_val, Y_val, config):
    # 1. Reproducibilidad para esta iteración
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Datos a tensores
    X_t = torch.from_numpy(X_tr).float().to(device)
    Y_t = torch.from_numpy(Y_train).long().to(device)
    X_v = torch.from_numpy(X_val).float().to(device)
    Y_v = torch.from_numpy(Y_val).long().to(device) # No necesitamos pesos aquí para la métrica, solo para el Loss
    
    # DataLoader
    dataset = TensorDataset(X_t, Y_t)
    loader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)
    
    # Modelo
    model = ImprovedMLP(input_size=X_tr.shape[1], output_size=2)
    model.to(device)
    
    # Optimizador y Loss
    # Usamos Focal Loss en lugar de pesos manuales (alpha maneja el balance)
    criterion = FocalLoss(alpha=0.75, gamma=2.0) 
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['wd'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=False)
    
    best_f1 = 0.0
    patience_counter = 0
    best_model_state = None
    
    print(f"--- Iniciando Semilla {seed} ---")
    
    for epoch in range(config['epochs']):
        epoch_loss = 0
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            
            # APLICAR MIXUP
            # Solo aplicamos mixup con cierta probabilidad (o siempre, prueba ambos)
            mixed_x, y_a, y_b, lam = mixup_data(xb, yb, alpha=0.4) # alpha 0.2 a 0.4 suele ir bien
            
            logits = model(mixed_x)
            
            # Usamos el criterio mixup
            loss = mixup_criterion(criterion, logits, y_a, y_b, lam)
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        # Validación
        model.eval()
        with torch.no_grad():
            logits_val = model(X_v)
            # Calculamos Loss para monitoreo
            val_loss = criterion(logits_val, Y_v).item()
            
            preds = torch.argmax(logits_val, dim=1).cpu().numpy()
            f1 = f1_score(Y_val, preds, average='macro')
            
        # Scheduler basado en F1 (queremos maximizar)
        scheduler.step(f1)
        
        # Guardado del mejor estado en RAM
        if f1 > best_f1:
            best_f1 = f1
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= config['patience']:
            break
            
    print(f"Semilla {seed} terminada. Mejor Val F1 Macro: {best_f1:.4f}")
    
    # Guardar modelo en disco para el ensamble
    torch.save(best_model_state, f"modelo_ensemble_seed_{seed}.pth")
    return best_f1

In [15]:
def entrenar_modelo_semilla(seed, X_tr, Y_train, X_val, Y_val, config):
    # 1. Reproducibilidad
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Datos a tensores
    X_t = torch.from_numpy(X_tr).float().to(device)
    Y_t = torch.from_numpy(Y_train).long().to(device)
    X_v = torch.from_numpy(X_val).float().to(device)
    Y_v = torch.from_numpy(Y_val).long().to(device)
    
    dataset = TensorDataset(X_t, Y_t)
    loader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True)
    
    # --- CAMBIO IMPORTANTE AQUÍ ---
    # Instanciamos la red de Fusión (GatedFusionMLP) en lugar del MLP simple.
    # Esta red ya sabe que la entrada es de 900 (300+300+300) internamente.
    model = GatedFusionMLP(output_size=2) 
    model.to(device)
    
    # Loss y Optimizador
    criterion = FocalLoss(alpha=0.75, gamma=2.0)
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['wd'])
    
    # Scheduler: Reduce el LR si el F1 no mejora
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)
    
    best_f1 = 0.0
    patience_counter = 0
    best_model_state = None
    
    print(f"--- Iniciando Semilla {seed} con GatedFusionMLP ---")
    
    for epoch in range(config['epochs']):
        model.train()
        
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            
            # NOTA: Si quisieras combinar Táctica 1 (Mixup) con Táctica 2,
            # aquí iría el bloque de mixup_data y mixup_criterion.
            # Por ahora, lo dejamos en entrenamiento estándar:
            logits = model(xb)
            loss = criterion(logits, yb)
            
            loss.backward()
            optimizer.step()
            
        # Validación
        model.eval()
        with torch.no_grad():
            logits_val = model(X_v)
            preds = torch.argmax(logits_val, dim=1).cpu().numpy()
            f1 = f1_score(Y_val, preds, average='macro')
            
        scheduler.step(f1)
        
        if f1 > best_f1:
            best_f1 = f1
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= config['patience']:
            break
            
    print(f"Semilla {seed} terminada. Mejor Val F1 Macro: {best_f1:.4f}")
    
    # Guardamos el modelo
    torch.save(best_model_state, f"modelo_ensemble_seed_{seed}.pth")
    return best_f1

In [16]:
def generar_semillas_fibonacci(cantidad, start_index=15):
    """
    Genera una lista de semillas basada en la secuencia de Fibonacci.
    
    Args:
        cantidad (int): Cuántas semillas necesitas (ej: 5 para tu ensamble).
        start_index (int): Cuántos pasos saltar al inicio para evitar números 
                           pequeños como 0, 1, 1, 2. Por defecto 15 empieza en 610.
    
    Returns:
        list: Lista de enteros listos para usar como semillas.
    """
    semillas = []
    a, b = 0, 1
    
    # 1. Avanzar la secuencia hasta el punto de inicio (offset)
    for _ in range(start_index):
        a, b = b, a + b
        
    # 2. Generar las semillas solicitadas
    for _ in range(cantidad):
        # Aplicamos modulo 2^32 - 1 para asegurar compatibilidad con numpy/torch
        # aunque Python soporta enteros infinitos, las librerias de ML a veces no.
        seed_val = a % (2**32 - 1)
        semillas.append(seed_val)
        
        # Siguiente paso de Fibonacci
        a, b = b, a + b
        
    return semillas

In [20]:
import os
# --- CONFIGURACIÓN GLOBAL ---
config = {
    'batch_size': 32,      # Bajamos un poco para regularizar
    'lr': 0.001,           # AdamW suele preferir LRs más bajos que 0.1
    'wd': 0.05,            # Weight Decay más fuerte para evitar overfitting
    'epochs': 100,
    'patience': 15
}

# Lista de semillas para el ensamble (puedes usar 3, 5 o 7)
seeds = generar_semillas_fibonacci(cantidad=5, start_index=8)

# 1. ENTRENAMIENTO DEL ENSAMBLE
f1_scores = []
for seed in seeds:
    score = entrenar_modelo_semilla(seed, X_tr, Y_train, X_val, Y_val, config)
    f1_scores.append(score)

print(f"\nPromedio F1 Macro (Validación) del Ensamble individual: {np.mean(f1_scores):.4f}")

# 2. INFERENCIA CON ENSAMBLE ACTUALIZADA (PARA TÁCTICA 2)
def predecir_ensamble(seeds, archivo_test_path):
    print("\n--- Iniciando Inferencia de Ensamble (Gated Fusion) ---")
    
    # Cargar datos de test
    df_test = pd.read_json(archivo_test_path, lines=True)
    we_ft_t = np.stack(df_test['we_ft'].values)
    we_mx_t = np.stack(df_test['we_mx'].values)
    we_es_t = np.stack(df_test['we_es'].values)
    
    # Concatenamos igual que en el train (N, 900)
    # La red GatedFusionMLP se encarga de separarlos internamente
    X_t_np = np.concatenate([we_ft_t, we_mx_t, we_es_t], axis=1)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X_test_tensor = torch.from_numpy(X_t_np).float().to(device)
    
    # Acumulador de probabilidades
    probs_totales = np.zeros((len(X_test_tensor), 2))
    
    # --- CAMBIO AQUÍ: Instanciar el modelo correcto ---
    model = GatedFusionMLP(output_size=2).to(device)
    
    for seed in seeds:
        nombre_archivo = f"modelo_ensemble_seed_{seed}.pth"
        # Cargar pesos
        model.load_state_dict(torch.load(nombre_archivo))
        model.eval()
        
        with torch.no_grad():
            logits = model(X_test_tensor)
            probs = F.softmax(logits, dim=1).cpu().numpy()
            probs_totales += probs
            
    # Promediar y sacar argmax
    probs_promedio = probs_totales / len(seeds)
    y_pred_final = np.argmax(probs_promedio, axis=1)
    
    return y_pred_final

# 3. GENERAR ARCHIVO
y_pred_test = predecir_ensamble(seeds, "../Datasets/dataset_humor_test_embeddings.json")

os.makedirs('resultados_focal_loss', exist_ok=True)

# Guardar
nombre_salida = "resultados_focal_loss/submission_ensemble_focal_loss_v5.csv"
df_out = pd.DataFrame({'klass': y_pred_test})
df_out['id'] = df_out.index + 1
df_out = df_out[['id', 'klass']]
df_out.to_csv(nombre_salida, index=False)

print(f"¡Archivo generado exitosamente: {nombre_salida}!")
print("Distribución de predicciones:", np.unique(y_pred_test, return_counts=True))

--- Iniciando Semilla 21 con GatedFusionMLP ---
Semilla 21 terminada. Mejor Val F1 Macro: 0.8009
--- Iniciando Semilla 34 con GatedFusionMLP ---


KeyboardInterrupt: 